Residential Building Permits
- Core Based Statistical Areas (CBSA)
- County
- Metropolitan Area (MA)
- Place


Currently - on our website: 
https://planning.maryland.gov/MSDC/Pages/newhh/newhh.aspx

- Monthly, Quarterly and Annual Reports
- A dropdown for each report type
- A list of reports, typically dozens of items long...
- Select and individual report
- That directs you to a new page, with 2-6 report documents that are downloadable as PSF of xlsx


Census 'Master Data' file: 
- https://www2.census.gov/econ/bps/Master%20Data%20Set/
- A 2.7GB .csv file
- BPS_Compiled_Fils_2024_09.csv etc
- 10.7 million rows
- When filtering for: MD, monthly reports for Counties, row count goes down to:
    - 4880 rows
    - A very manageable dataset

Monthly County Data files:
    - https://www2.census.gov/econ/bps/County/
    - files ending with 'c' are 'current month'
    - files ending with 'y' are 'year to date'

Census Visualization:
    https://www.census.gov/construction/bps/visualizations/datatool/index.html
    

Most Ambitious Ideas:
- Store all of the building permit data in an SQL database at the Department of Planning
- Have a browser-based application that an queries the database
    - Visualizations (filter based on time)
    - Data Exports
    - Allow comparisons between two 'user-defined' periods of time
- Some that integrates 'seemlessly' with our website - like what? - Can we 'draw' the best solution and work backwards?
- Include some FRED time Series data - interest rate

Less Ambitious Idea:
- With only 5k rows, we can make a Tableau dashboard for this right now
- We are only adding 24 rows per month if we just do counties
- No need for 'reports' every month - everything is in the dashboard


Questions:
- In the current reports, Places and Counties (+BC) are included - are the county counts inclusive of the Places? Ie. does Talbot county include Easton?
    

In [1]:
import pandas as pd

permit_data = pd.read_csv('BPS_Compiled_File_202409.csv', encoding='latin1')

for i in permit_data.columns:
    print(i)

# print the number of rows and columns in the dataset
print(permit_data.shape)

# print unique values in the 'STATE_CODE' column
# print(permit_data['STATE_NAME'].unique())


C:\Users\tschlachter\AppData\Local\Temp\ipykernel_20312\2372355642.py:3: DtypeWarning: Columns (9,11,13,16,24,28,32,35,60) have mixed types. Specify dtype option on import or set low_memory=False.
  permit_data = pd.read_csv('BPS_Compiled_File_202409.csv', encoding='latin1')


BLDGS_1_UNIT
BLDGS_1_UNIT_REP
BLDGS_2_UNITS
BLDGS_2_UNITS_REP
BLDGS_3_4_UNITS
BLDGS_3_4_UNITS_REP
BLDGS_5_UNITS
BLDGS_5_UNITS_REP
CBSA_CODE
CBSA_NAME
CENSUS_PLACE_CODE
CENTRAL_CITY
COUNTY_CODE
COUNTY_NAME
CSA_CODE
DIVISION_CODE
DIVISION_NAME
FILE_NAME
FIPS_COUNTY_5_DIGITS
FIPS_MCD_CODE
FIPS_PLACE_CODE
FOOTNOTE_CODE
ID_6_DIGIT
LOCATION_TYPE
MONCOV
MONTH
NUMBER_OF_MONTHS_REP
PERIOD
PLACE_NAME
UNIQUE_PLACE_ID
POP
REGION_CODE
REGION_NAME
SOURCE_CODE
STATE_CODE
STATE_NAME
SURVEY_DATE
TOTAL_BLDGS
TOTAL_BLDGS_REP
TOTAL_UNITS
TOTAL_UNITS_REP
TOTAL_VALUE
TOTAL_VALUE_REP
UNITS_1_UNIT
UNITS_1_UNIT_REP
UNITS_2_UNITS
UNITS_2_UNITS_REP
UNITS_3_4_UNITS
UNITS_3_4_UNITS_REP
UNITS_5_UNITS
UNITS_5_UNITS_REP
VALUE_1_UNIT
VALUE_1_UNIT_REP
VALUE_2_UNITS
VALUE_2_UNITS_REP
VALUE_3_4_UNITS
VALUE_3_4_UNITS_REP
VALUE_5_UNITS
VALUE_5_UNITS_REP
YEAR
ZIP_CODE
LOCATION_NAME
(10716539, 62)


In [2]:
print(permit_data['LOCATION_TYPE'].unique())

['Place' 'County' 'State' 'Region' 'Division' 'Country' 'Metro' 'Micro']


In [19]:
def md_permits(permit_data):
    locations = ['Place', 'County', 'Metro', 'Region']
    md_permits = permit_data[permit_data['STATE_NAME'].isin(['Maryland', 'MD'])]
    md_permits

    # sort the data by the 'YEAR' then by 'MONTH' then by 'COUNTY_CODE'
    md_permits = md_permits.sort_values(by=['YEAR', 'MONTH', 'COUNTY_CODE'])

    # filter out rows where 'MONTH' is not in the range 1 to 12 - check how many rows are removed
    md_permits = md_permits[md_permits['MONTH'].between(1, 12)]

    # combine month and year columns into a 'date' column
    md_permits['date'] = pd.to_datetime(md_permits['YEAR'].astype(str) + '-' + md_permits['MONTH'].astype(str))

    # drop all rows where 'PERIOD' is not equal to Monthly
    md_permits = md_permits[md_permits['PERIOD'] == 'Monthly']

    for l in locations:
        x = md_permits[md_permits['LOCATION_TYPE'] == l]
        x.to_csv('md_permits_monthly_{}.csv'.format(l), index=False)

def national_permits(permit_data):
    locations = ['Metro', 'Region']
    # filter out rows where 'MONTH' is not in the range 1 to 12 - check how many rows are removed
    x = permit_data[permit_data['MONTH'].between(1, 12)]

    # combine month and year columns into a 'date' column
    x['date'] = pd.to_datetime(x['YEAR'].astype(str) + '-' + x['MONTH'].astype(str))

    # drop all rows where 'PERIOD' is not equal to Monthly
    x = x[x['PERIOD'] == 'Monthly']

    for l in locations:
        y = x[x['LOCATION_TYPE'] == l]
        if l == 'Metro':
            # filter out row where 'CBSA_NAME' does not contain 'MD' with Caps
            y = y[y['CBSA_NAME'].str.contains('MD', case=True)]
        y.to_csv('permits_monthly_{}.csv'.format(l), index=False)

In [20]:
# md_permits(permit_data)
national_permits(permit_data)

C:\Users\tschlachter\AppData\Local\Temp\ipykernel_20312\3215156790.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['date'] = pd.to_datetime(x['YEAR'].astype(str) + '-' + x['MONTH'].astype(str))


In [18]:
# from permits monthly csv, print unique values of column CBSA_NAME
z = pd.read_csv('permits_monthly_Metro.csv')

for i in permit_data['CBSA_NAME'].unique():
    print(i)

C:\Users\tschlachter\AppData\Local\Temp\ipykernel_20312\1668136787.py:2: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  z = pd.read_csv('permits_monthly_Metro.csv')


nan
Indianapolis-Carmel-Anderson  IN
Iowa City  IA
Ithaca  NY
Jackson  MI
Jackson  MS
Lubbock  TX
Lynchburg  VA
Macon-Bibb County  GA
Jackson  TN
Jacksonville  FL
Jacksonville  NC
Janesville-Beloit  WI
Jefferson City  MO
Johnson City  TN
Madera  CA
Madison  WI
Manchester-Nashua  NH
Manhattan  KS
Mankato  MN
Mansfield  OH
McAllen-Edinburg-Mission  TX
Omaha-Council Bluffs  NE-IA
Orlando-Kissimmee-Sanford  FL
Oshkosh-Neenah  WI
Owensboro  KY
Oxnard-Thousand Oaks-Ventura  CA
Palm Bay-Melbourne-Titusville  FL
Johnstown  PA
Jonesboro  AR
Joplin  MO
Kahului-Wailuku-Lahaina  HI
Kalamazoo-Portage  MI
Kankakee  IL
Medford  OR
Memphis  TN-MS-AR
Merced  CA
Miami-Fort Lauderdale-Pompano Beach  FL
Michigan City-La Porte  IN
Midland  MI
Panama City  FL
Parkersburg-Vienna  WV
Pensacola-Ferry Pass-Brent  FL
Peoria  IL
Philadelphia-Camden-Wilmington  PA-NJ-DE-MD
Phoenix-Mesa-Chandler  AZ
Pine Bluff  AR
Gettysburg  PA
Glens Falls  NY
Hammond  LA
Hanford-Corcoran  CA
Harrisburg-Carlisle  PA
Harrisonburg  